In [1]:
import selenium

In [2]:
search_web_url = 'http://etds.lib.ncku.edu.tw/etdservice/searchform'

In [166]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
def searchPaperData(student_name):
    #selectnium part
    driver = webdriver.Chrome('/Users/syushengwei/Desktop/chromedriver') 
    driver.get("http://etds.lib.ncku.edu.tw/etdservice/searchform")
    driver.find_element_by_name("query_word1").send_keys(student_name)
    Select(driver.find_element_by_name("query_field1")).select_by_visible_text(u"研究生")
    driver.find_element_by_name("Submit").click()
    html = driver.page_source 
    driver.close()
    
    #bf4 part
    soup = BeautifulSoup(html, 'html.parser')
    if '查無相關資料' in soup.text:
        return ['0\t'+student_name+'\t'+'']
    else:
        Paper_List = []
        if_find = 0
        for form in soup.find_all('form',{"name":"list_form"}):
            for tr in form.find_all('tr'):
                if 'EMBA' in tr.text:
                    if_find = 1
                    Get_Paper_Data = []  
                    for td in tr.find_all('td'):
                        if td.text == '':
                            continue
                        if '\n       成功大學/管理學院/高階管理碩士在職專班(EMBA)' in td.text:
                            paper_name = td.text.split('\n       成功大學/管理學院/高階管理碩士在職專班(EMBA)')
                            Get_Paper_Data.append(paper_name[0].replace('\n','').replace(' ',''))
                            Get_Paper_Data.append('成功大學/管理學院/高階管理碩士在職專班(EMBA)')
                            Get_Paper_Data.append(paper_name[1])
                        else:
                            Get_Paper_Data.append(td.text)
                    Paper_List.append('\t'.join(Get_Paper_Data))
        if if_find == 1 :
            return Paper_List
        else:
            return ['0\t'+student_name+'\t'+'']

In [111]:
import codecs 
Student_Dic = {}
Same_Name_List = []
Same_Name_Info = []
with codecs.open('EMBA學號.csv','r',encoding= 'utf8')as infile:
    Data_Line = infile.readlines()

In [112]:
    for line in Data_Line:
        #print(line.split('\t'))
        data = line.replace('\r\n','').split('\t')
        if Student_Dic.get(data[1],None) == None:
            Student_Dic.update({data[1]:data[0]})
        else:
            Same_Name_List.append(data[1])
            Same_Name_Info.append([data[0],data[1]])
            if [Student_Dic.get(data[1]), data[1]] not in Same_Name_Info:
                Same_Name_Info.append([Student_Dic.get(data[1]), data[1]])

In [118]:
EMBA_Papers = []
for student in Student_Dic:
    try:
        for paper in searchPaperData(student):
            if student in Same_Name_List:
                EMBA_Papers.append(''+'\t'+paper)
            else:
                EMBA_Papers.append(Student_Dic[student]+'\t'+paper)
    except Exception as e:
        print(student,e)

In [225]:
#print(Same_Name_List)

In [120]:
print(len(EMBA_Papers))

1424


In [117]:
len(Student_Dic)

1492

In [224]:
#with codecs.open('EMBA論文','w',encoding='utf8') as outfile:
#    for line in EMBA_Papers:
#        outfile.write(line+'\n')

In [223]:
#with codecs.open('EMBA論文','r',encoding='utf8') as infile:
     #EMBA_Papers = infile.readlines()